In [49]:
import glob
from datetime import timedelta, datetime
import os
from shutil import rmtree
import csv 
import pandas as pd
import zipfile

# Comput end times
Provided with the data file directory, compute all the proper end times given a custom time shift.

In [50]:
# arguments 
data_dir = '/well/doherty/users/cxx579/project_data/test_raine'
timeshift = timedelta(days=7)
script_dir = '/well/doherty/users/cxx579/test_process/myBBAA/process-raine-cmds.txt'
dest_script_dir = '/well/doherty/users/cxx579/test_process/myBBAA/process-raine-cmds-final.txt'

In [37]:
endTimeFilePath = data_dir + '/endTime.csv'
file_desp = data_dir + '/*.gt3x'
tmp_dir = data_dir + '/tmp'
info_txt_dir = tmp_dir + '/info.txt'
file_list = glob.glob(file_desp)
if os.path.exists(tmp_dir):
    print(tmp_dir + " exists! Removing....")
    rmtree(tmp_dir) 
os.mkdir(tmp_dir)

/well/doherty/users/cxx579/project_data/test_raine/tmp exists! Removing....


In [38]:
def GT3XfromTickToMillisecond(mydate):
    mydate = mydate - 621355968000000000
    mydate /= 10000000
    mydate = int(mydate)
    return datetime.utcfromtimestamp(mydate)

In [39]:
# get the start time 
sample_file_path = file_list[0]

In [40]:
def obtain_start_date(txt_dir):
    
    f = open(txt_dir, "r")
    start_date = "0"
    for x in f:
        substrs = x.split(": ")
        if (substrs[0] == 'Start Date'):
            start_date = substrs[1]
            start_date = start_date[:-1] # remove line break'
            return int(start_date)
        
    return int(start_date)

In [41]:
startTimes = []
endTimes = []
fileNames = []

for current_file in file_list:
    with zipfile.ZipFile(current_file, 'r') as zip_ref:
        zip_ref.extractall(tmp_dir)
    
    start_date = obtain_start_date(info_txt_dir)
    localDateTime = GT3XfromTickToMillisecond(start_date)
    endTime = localDateTime + timeshift
    startTime = localDateTime.strftime('%Y-%m-%dT%H:%M')
    endTime = endTime.strftime('%Y-%m-%dT%H:%M')
    fileName = current_file.split('/')[-1]
    
    fileNames.append(fileName)
    startTimes.append(startTime)
    endTimes.append(endTime)

In [42]:
# Exporting end times to csv
if os.path.exists(endTimeFilePath):
    os.remove(endTimeFilePath)
    
with open(endTimeFilePath, mode='w') as csv_file:
    fieldnames = ['file_name', 'startTime', 'endTime']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()

    for i in range(len(fileNames)):
        writer.writerow({'file_name': fileNames[i], 
                         'startTime': startTimes[i], 
                         'endTime': endTimes[i]})    

# Modify command line script 
Input for this one should be the script paths and endTime table. 
We will add the adaptive endTime to each file.

In [51]:
endTimeDf = pd.read_csv(endTimeFilePath)

In [52]:
endTimeDf

,file_name,startTime,endTime
0,770514_wrist_2016-01-18.gt3x,2016-01-18T18:00,2016-01-25T18:00
1,770768_wrist_2016-04-12.gt3x,2016-04-12T18:00,2016-04-19T18:00
2,770757_wrist_2017-04-11.gt3x,2017-04-11T18:00,2017-04-18T18:00


In [55]:
def extract_file_name(command_str):
    file_path = command_str.split(' ')
    file_path = file_path[2]
    file_path = file_path[1:]
    file_path = file_path[:-1] # remove quotation marks 
    
    return file_path.split("/")[-1]

In [56]:
f = open(script_dir, "r")
k = 0

if os.path.exists(dest_script_dir):
    os.remove(dest_script_dir)

with open(dest_script_dir, 'a') as dest_f:
    for x in f:
        my_file_name = extract_file_name(x)
        my_end_time = endTimeDf.loc[endTimeDf['file_name'] == my_file_name]['endTime'].item()
        x = x[:-1] # remove line break    
        x = x + ' --endTime ' + my_end_time + '\n'
        dest_f.write(x)
        
f.close()